In [1]:
import torch
major_version, minor_version = torch.cuda.get_device_capability()

In [5]:
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-calqj65_/unsloth_9d13e8730eab4513a31eb0aaa49ebaa3
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-calqj65_/unsloth_9d13e8730eab4513a31eb0aaa49ebaa3
  Resolved https://github.com/unslothai/unsloth.git to commit ec19e61c854dcf9104386fa63fc6c4f2944d4f35
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 4.0 MB/s eta 0:00:00 MB/s eta 0:00:01:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 8.1 MB/s eta 0:00:009.0 MB/s eta 0:00:01
  Created wheel for unsloth: filename=unsloth-2024.4-py3-none-any.whl size=99172 sha256=9c29387312ce46aef5545a4486ab8d84bbfae34d2b4120f0d367a38bed8dff38
  Stored in directory: /tmp/pip-ephem-wheel-cache-aliy97c6/wheels/ed/d4/e9/76fb290ee3df0a5

In [6]:
!pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes

  Using cached einops-0.7.0-py3-none-any.whl.metadata (13 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 5.7 MB/s eta 0:00:000:00:01m eta 0:00:01
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 8.6 MB/s eta 0:00:009.8 MB/s eta 0:00:01
Using cached einops-0.7.0-py3-none-any.whl (44 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 MB 2.7 MB/s eta 0:00:00m eta 0:00:010:00:03
  Created wheel for flash-attn: filename=flash_attn-2.5.8-cp310-cp310-linux_x86_64.whl size=121735960 sha256=f7b1002a954d42f6451c812d1ab69a0f70092e447df49b6dd384b2006f9094ee
  Stored in directory: /root/.cache/pip/wheels/9b/5b/2b/dea8af4e954161c49ef1941938afcd91bb93689371ed12a226
Successfully built flash-attn


## inference

In [2]:
from transformers import TextStreamer
from unsloth import FastLanguageModel

/workspace/pyenvs/LLM_ENV/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.2.2+cu121 with CUDA 1201 (you have 2.1.0+cu118)
    Python  3.10.14 (you have 3.10.13)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details


In [3]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "./models/checkpoint-500",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    device_map = "auto",)

FastLanguageModel.for_inference(model) # Enable native 2x faster inference

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 3060. Max memory: 11.755 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.1.0+cu118. CUDA = 8.6. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
alpaca_prompt = """
### 질문:
{}

### 답변:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "롱브릭타일은 어떤 종류의 타일인가요? 그리고 페인트 상도재의 역할과 사용 방법에 대해 알려주세요.", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, 
                   streamer = text_streamer, 
                   max_new_tokens = 512, 
                   top_p = 0.9,
                   top_k = 50,
                   repetition_penalty=1.2,
                   early_stopping=True, 
                   do_sample=True, 
                   eos_token_id=tokenizer.eos_token_id)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>
### 질문:
롱브릭타일은 어떤 종류의 타일인가요? 그리고 페인트 상도재의 역할과 사용 방법에 대해 알려주세요.

### 답변:
롱브릭타일에 대해서 설명드리겠습니다. 롱브릭타일은 브레킷형태로 만들어진 벽지입니다. 이는 바닥이나 천장, 외벽 또는 내벽 등 다양한 표면에서 용이하게 사용할 수 있는 점이 특징으로 꼽을 수 있습니다. 이러한 유형의 벽지는 주변 구조물과의 미세한 차이를 보완하여 균형과 형태를 유지하는 데 도움을 줍니다. 또한 롱브릿크타이는 높은 강도의 재료와 함께 사용되므로 건축자재로서 널리 활용됩니다.
페인트상도재의 주요 기능은 그 이름대로 페인팅 시 필요한 도구가 되는 것입니다. 이 과정에서는 필름 등의 매개체를 이용해 색칠된 패턴이나 기하학적인 그림 등을 제작하고, 또한 금속, 유기 물질 등 여러 가지 재료로부터 얻어내는 색을 통해 독특하고 부드럽게 표현될 수 있도록 합니다. 이러한 요소들이 모두 결합되어 창작적 결과물을 낳습니다. 따라서 페인트상도재는 작품의 감성 조직 뿐만 아니라 공간의 디자인에도 중요한 영향을 끼칩니다.<|end_of_text|>


In [12]:
alpaca_prompt = """
### 질문:
{}

### 답변:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "공동주택의 실내 공기질을 측정할 때, 어떤 층에서 샘플을 채취하는 것이 가장 적절한가요?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, 
                   streamer = text_streamer, 
                   max_new_tokens = 512, 
                   early_stopping=True, 
                   top_p = 0.9,
                   top_k = 50,
                   repetition_penalty=1.2,
                   do_sample=True, 
                   eos_token_id=tokenizer.eos_token_id)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>
### 질문:
공동주택의 실내 공기질을 측정할 때, 어떤 층에서 샘플을 채취하는 것이 가장 적절한가요?

### 답변:
공동 주거 건물에서는 전층 차별적이고 균형 있는 관리를 위해 공동 사용 부서(엘리베이터 기계실 등)에 대해서는 미측하고, 단위 집합에 한해 분포도상 차이가 크지 않으므로 특정이 없는 경우 2~4층의 층에서의 데이터 수집이 권장됩니다. 다만, 공동 소유 구조인 경우에는 상업시설이 포함된 고층층과 저밀용간소주의 인테리어로 인하여 층 별로 영향을 받는다면 해당층으로부터 샘플을 추출해야 합니다. 만약 각 층별로 구분해서 데이터 분석을 하고 싶다면, 엘리베이터 기계실은 제외하며 다른 층들로부터 각각 샘플을 추출합니다.

식품가구 및 가산수지는 비점제적 방법을 통해 하중 배치를 하는 것을 고려해야 합니다. 또한, 공급부등을 중심으로 공간 중립화 요소를 통하게 함으로써, 관측 값을 조건 변수로 설정하고 계수를 산정하는데 도움을 얻을 수 있습니다.
<|end_of_text|>


In [18]:
alpaca_prompt = """
### 질문:
{}

### 답변:
{}"""

inputs = tokenizer(
[
    alpaca_prompt.format(
        "새집증후군의 주요 원인은 무엇인가요?", # input
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, 
                   streamer = text_streamer, 
                   max_new_tokens = 512, 
                   early_stopping=True, 
                   top_p = 0.92,
                   top_k = 50,
                   repetition_penalty=1.2,
                   do_sample=True, 
                   eos_token_id=tokenizer.eos_token_id)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>
### 질문:
새집증후군의 주요 원인은 무엇인가요?

### 답변:
새집이 가지고 있는 아토피 유전자, 새로 제조된 소재 등 다양한 요인으로 인해 발생할 수 있습니다. 또한 내화제나 발수제를 포함한 화학 물질에 의해 영향을 받을 수도 있습니다.
따라서 새집에서 신중하게 선택하여 사용하고, 통풍을 통해 오염물질 및 미세먼지를 배출하는 것이 중요합니다.

아마도 새집과의 상관관계로는 다음과 같은 몇 가지 사항들을 고려해야 합니다.
* 시간: 새벽이나 저녁에는 더 건기적이며 밀실하다는 점과 관련되어, 특정 시점에서는 강도가 높아 위험성이 커진다.
* 온도의 변화: 특히 겨울철에는 새벽 혹은 저녁과 같은 시간대에 새벽 기온이 -1˚C부터 +7˚C까지 떨어뜨려 위력이 낮지만 여름철처럼 온도를 급격히 바꾸면 방전현상이 증가한다는 것입니다. 따라서 온도의 변화를 피하면서 건폐율 0.2 이상의 공간에서는 전원 공급 중단시에는 무손실 복원을 할 수 있도록 하는 시스템을 도입하세요.
<|end_of_text|>
